In [1]:
import numpy as np
import time

import PIL.Image as Image
import matplotlib.pylab as plt

import tensorflow as tf
import tensorflow_hub as hub

import datetime

%load_ext tensorboard

In [2]:
data_root_train = "../data/animals/elephants_data/"
data_root_test = "../data/animals/elephants_data/"

In [3]:
batch_size = 64
img_height = 224
img_width = 224

train_ds = tf.keras.utils.image_dataset_from_directory(
  str(data_root_train),
  seed=123,
  color_mode="grayscale"
  
  image_size=(img_height, img_width),
  batch_size=batch_size
)

Found 4681 files belonging to 2 classes.


In [4]:
class_names = np.array(train_ds.class_names)
print(class_names)

['elephant' 'others']


In [5]:
AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [6]:
for image_batch, labels_batch in train_ds:
  print(image_batch.shape)
  print(labels_batch.shape)
  break

(64, 224, 224, 3)
(64,)


In [7]:
mobilenet_v2 = "https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4"
inception_v3 = "https://tfhub.dev/google/tf2-preview/inception_v3/feature_vector/4"

feature_extractor_model = mobilenet_v2

In [8]:
num_classes = len(class_names)

model = tf.keras.Sequential([
  tf.keras.layers.Conv2D(filters=32, kernel_size=(3,3), strides=(1, 1), padding="valid", activation="relu", input_shape=( 224, 224, 3)),
  tf.keras.layers.MaxPooling2D((2, 2)),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(64),
  tf.keras.layers.Dense(16),
  tf.keras.layers.Dense(num_classes)
])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 222, 222, 32)      896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 111, 111, 32)      0         
_________________________________________________________________
flatten (Flatten)            (None, 394272)            0         
_________________________________________________________________
dense (Dense)                (None, 64)                25233472  
_________________________________________________________________
dense_1 (Dense)              (None, 16)                1040      
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 34        
Total params: 25,235,442
Trainable params: 25,235,442
Non-trainable params: 0
____________________________________________

In [9]:
model.compile(
  optimizer=tf.keras.optimizers.Adam(),
  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    # loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
  metrics=['acc'])

log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

In [10]:
NUM_EPOCHS = 5

history = model.fit(train_ds, epochs=NUM_EPOCHS)

Epoch 1/5
74/74 [==============================] - 62s 810ms/step - loss: 4554.5122 - acc: 0.7761
Epoch 2/5
74/74 [==============================] - 61s 829ms/step - loss: 35.8251 - acc: 0.9135
Epoch 3/5
74/74 [==============================] - 56s 755ms/step - loss: 12.8844 - acc: 0.9457
Epoch 4/5
74/74 [==============================] - 57s 767ms/step - loss: 4.9285 - acc: 0.9690
Epoch 5/5
74/74 [==============================] - 55s 748ms/step - loss: 2.7020 - acc: 0.9697


In [11]:
predicted_batch = model.predict(image_batch)
predicted_id = tf.math.argmax(predicted_batch, axis=-1)
predicted_label_batch = class_names[predicted_id]
print(predicted_label_batch)

['elephant' 'elephant' 'elephant' 'elephant' 'others' 'others' 'others'
 'others' 'elephant' 'others' 'elephant' 'elephant' 'elephant' 'others'
 'others' 'elephant' 'elephant' 'others' 'others' 'elephant' 'others'
 'others' 'elephant' 'elephant' 'elephant' 'others' 'others' 'elephant'
 'others' 'others' 'others' 'elephant' 'elephant' 'elephant' 'others'
 'elephant' 'elephant' 'others' 'others' 'elephant' 'elephant' 'elephant'
 'elephant' 'elephant' 'others' 'others' 'others' 'elephant' 'elephant'
 'others' 'elephant' 'elephant' 'elephant' 'elephant' 'others' 'elephant'
 'others' 'others' 'others' 'elephant' 'others' 'elephant' 'elephant'
 'others']


In [12]:
t = time.time()

export_path = "../models/animals_road_scratch_v2_{}".format(int(t))
model.save(export_path)

export_path

INFO:tensorflow:Assets written to: ../models/animals_road_scratch_v2_1640764809\assets


'../models/animals_road_scratch_v2_1640764809'

In [13]:
def toTflite(model, filename):
  converter = tf.lite.TFLiteConverter.from_keras_model(model)
  tfliteModel = converter.convert()
  open (f"{filename}.tflite" , "wb") .write(tfliteModel)

In [14]:
toTflite(model, export_path)

INFO:tensorflow:Assets written to: C:\Users\user\AppData\Local\Temp\tmpa2ezy82f\assets
